In [8]:
import sys

sys.path.append("..")

from src.metrics import precision_at_k, recall_at_k
from src.metrics import precision_at_k
from src.utils import prefilter_items
from src.recommenders import MainRecommender

In [9]:
# disabling warnings entirely
import warnings
warnings.filterwarnings('ignore')

In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
!pip install lightfm
# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight
import  implicit
from lightfm import LightFM
from lightfm.evaluation import precision_at_k, recall_at_k

# Функции из 1-ого вебинара
import os

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

In [15]:
data = pd.read_csv(r'C:\Users\nikita.saprykin\Desktop\RCS\урок_5\retail_train.csv')

data = data.sample(2000, random_state = 2021) # ограничение по записям 2000


item_features = pd.read_csv(r'C:\Users\nikita.saprykin\Desktop\RCS\урок_5\product.csv')
user_features = pd.read_csv(r'C:\Users\nikita.saprykin\Desktop\RCS\урок_5\hh_demographic.csv')

# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)

# train test split
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

## 1. Filter items

In [ ]:
n_items_before = data_train['item_id'].nunique()

#TO SPPED IT UP (INCREASE in PRODUCT)
#TO SPPED IT UP (INCREASE in PRODUCT)
#TO SPPED IT UP (INCREASE in PRODUCT)
#TO SPPED IT UP (INCREASE in PRODUCT)
#TO SPPED IT UP (INCREASE in PRODUCT)
take_n_popular = 1000
#TO SPPED IT UP (INCREASE in PRODUCT)
#TO SPPED IT UP (INCREASE in PRODUCT)
#TO SPPED IT UP (INCREASE in PRODUCT)
#TO SPPED IT UP (INCREASE in PRODUCT)
#TO SPPED IT UP (INCREASE in PRODUCT)
#TO SPPED IT UP (INCREASE in PRODUCT)


data_train_filtered = prefilter_items(data_train, take_n_popular=take_n_popular, item_features=item_features)

n_items_after = data_train_filtered['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

## 2. Prepare data set

## 2.1 Prepare csr TRAIN matrix

In [18]:
train_user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

train_user_item_matrix = train_user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат sparse matrix
sparse_train_user_item = csr_matrix(train_user_item_matrix).tocsr()

## 2.2 Prepare csr TEST matrix

In [19]:
data_test = data_test[data_test['item_id'].isin(data_train['item_id'].unique())]

test_user_item_matrix = pd.pivot_table(data_test, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

test_user_item_matrix = test_user_item_matrix.astype(float) # необходимый тип матрицы для implicit

sparse_test_user_item = csr_matrix(test_user_item_matrix).tocsr()

In [20]:
id_to_itemid, id_to_userid, itemid_to_id, userid_to_id =\
MainRecommender.prepare_dicts(train_user_item_matrix)

## 3. Prepare user and item features

In [21]:
user_feat = pd.DataFrame(train_user_item_matrix.index)
user_feat = user_feat.merge(user_features, on='user_id', how='left')
user_feat.set_index('user_id', inplace=True)

In [22]:
item_feat = pd.DataFrame(train_user_item_matrix.columns)
item_feat = item_feat.merge(item_features, on='item_id', how='left')
item_feat.set_index('item_id', inplace=True)

## Encoding features

In [23]:
user_feat_lightfm = pd.get_dummies(user_feat, columns=user_feat.columns.tolist())
item_feat_lightfm = pd.get_dummies(item_feat, columns=item_feat.columns.tolist())

## Preparing input data for the LifgtFM

In [24]:
user_features=csr_matrix(user_feat_lightfm.values).tocsr()
item_features=csr_matrix(item_feat_lightfm.values).tocsr()

sample_weight=coo_matrix(train_user_item_matrix)

## Init model

In [25]:
model = LightFM(no_components=40,
#                 loss='bpr',
                loss='warp',
                learning_rate=0.05, 
                item_alpha=0.1,
                user_alpha=0.1, 
                random_state=42)

## Train

In [26]:
model.fit((sparse_train_user_item > 0) * 1,  # user-item matrix из 0 и 1
          sample_weight=sample_weight,
          user_features=user_features,
          item_features=item_features,
          epochs=15, 
          num_threads=4,
          verbose=False)

## Evaluation

In [27]:
user_emb = model.get_user_representations(features=csr_matrix(user_feat_lightfm.values).tocsr())

In [28]:
item_emb = model.get_item_representations(features=csr_matrix(item_feat_lightfm.values).tocsr())

## Train precision

In [29]:
train_precision = precision_at_k(model,
                                 sparse_train_user_item, 
                                 user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                                 item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                                 k=5).mean()

train_precision

0.011275415

In [30]:
test_item_ids = np.array([1, 2, 3, 200, 1200])

predictions = model.predict(user_ids=0,
                            item_ids=test_item_ids,
                            user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                            item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                            num_threads=4)
predictions

array([-3.0927978, -3.0772839, -2.9638767, -2.770964 , -2.99987  ],
      dtype=float32)

## Test precision

In [31]:
test_precision = precision_at_k(model, 
                                sparse_test_user_item, 
                                user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                                item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                                k=5).mean()

test_precision

0.03529412

# Домашнее задание

1) Прочитать статьи про BPR, WARP loss, learning-to-rank*

2) Сделать грид серч текущей модели

In [32]:
from itertools import product

def dict_configs(d):
    for vcomb in product(*d.values()):
        yield dict(zip(d.keys(), vcomb))

param_grid = {
            "no_components": np.arange(25,55,25),
#             "learning_schedule": ["adagrad"], #"adadelta"
            "loss": ["warp"], #"bpr"
            "learning_rate": np.arange(.01,.1,.05),
            "item_alpha": np.arange (.01,.1,.05),
            "user_alpha": np.arange (.01,.1,.05),
#             "max_sampled": np.arange(1,101,25),
            "num_epochs": np.arange(10,70,30),
            "random_state":[42],
        }

def grid_search(sparce_matrix_train, 
                  test, 
                  sample_weight, 
                  user_features,
                  item_features,
                num_threads=8):

    for hyperparams in dict_configs(param_grid):
        num_epochs = hyperparams.pop("num_epochs")
       
        model = LightFM(**hyperparams)
        
        model.fit(sparse_train_user_item,
                  sample_weight=sample_weight,
                  user_features=user_features,
                  item_features=item_features,
                  epochs=num_epochs, 
                  num_threads=num_threads,
                  verbose=False)

        score = precision_at_k(model,
                               test,
                               user_features=user_features,
                               item_features=item_features,
                               k=5).mean()

        hyperparams["num_epochs"] = num_epochs

        yield (score, hyperparams, model)

In [33]:
from lightfm.datasets import fetch_movielens
(score, hyperparams, model) = max(grid_search(sparse_train_user_item, 
                                                sparse_test_user_item,
                                                sample_weight=sample_weight,
                                                user_features=user_features,
                                                item_features=item_features,
                                             ),
                                  key=lambda x: x[0])

print("Best score {} at {}".format(score, hyperparams))

Best score 0.052941177040338516 at {'no_components': 50, 'loss': 'warp', 'learning_rate': 0.060000000000000005, 'item_alpha': 0.01, 'user_alpha': 0.01, 'random_state': 42, 'num_epochs': 40}
